```
Team Name: RAV
Team Members: VIGNESH J, ASHWATH VINODKUMAR, RAHUL BHARGAV TALLADA
Leaderboard Rank: 50
```

# Soil Classification Challenge - Inference Notebook

This notebook contains the inference pipeline for the soil classification challenge. It loads the trained model and makes predictions on test images using a hybrid approach combining zero-shot CLIP features and a trained logistic regression classifier.

## 1. Setup and Dependencies

First, we'll install the required packages and import necessary libraries.

In [ ]:
!pip install open-clip-torch pandas pillow scikit-learn --quiet

import open_clip
import torch
import pandas as pd
import numpy as np
import pickle
from PIL import Image
from pathlib import Path

## 2. Configuration

Define the model configuration and parameters.

In [ ]:
# Configuration (MODIFY IF NEEDED)
MODEL_NAME = "ViT-H-14"        # High-performance vision transformer
PRETRAINED = "laion2b_s32b_b79k"  # Pretraining dataset
BATCH_SIZE = 8                 # Reduce to 4 if OOM errors persist
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLASSES = ["Alluvial soil", "Black Soil", "Clay soil", "Red soil"]

print(f"Using device: {DEVICE}")

## 3. Model Loading

Load the pre-trained CLIP model and preprocessor.

In [ ]:
# Load model and preprocessor
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name=MODEL_NAME,
    pretrained=PRETRAINED
)
model = model.to(DEVICE).eval()
print(f"Loaded {MODEL_NAME} model with {PRETRAINED} weights")

## 4. Load Trained Model and Embeddings

Load the classifier and text embeddings saved during training.

In [ ]:
# Load classifier
with open('../models/classifier.pkl', 'rb') as f:
    clf = pickle.load(f)
    
# Load text embeddings
text_embeddings = torch.load('../models/text_embeddings.pt')

print("Loaded trained classifier and text embeddings")

## 5. Load Test Data

Load the test data for inference.

In [ ]:
# Load test data
# Update paths as needed for your environment
test_df = pd.read_csv("/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv")
print(f"Loaded {len(test_df)} test samples")
print(test_df.head())

## 6. Image Embedding Function

Define a memory-optimized function to generate image embeddings in batches.

In [ ]:
# Memory-optimized embedding generator
def get_image_embeddings(image_paths):
    """Batch processing to prevent OOM errors"""
    embeddings = []
    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch = torch.stack([preprocess(Image.open(p).convert("RGB")) for p in batch_paths])
        
        with torch.no_grad():
            batch = batch.to(DEVICE)
            batch_emb = model.encode_image(batch)
            embeddings.append(batch_emb.cpu().numpy())
        
        # Explicit memory cleanup
        del batch, batch_emb
        torch.cuda.empty_cache()
    
    return np.concatenate(embeddings)

## 7. Hybrid Prediction Function

Define a function that combines logistic regression predictions with zero-shot CLIP predictions.

In [ ]:
# Hybrid prediction function
def predict_image(image_path):
    img_emb = get_image_embeddings([image_path])
    probe_pred = clf.predict_proba(img_emb)
    
    image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        
        zero_shot_probs = []
        for cls in CLASSES:
            text_features = text_embeddings[cls].to(DEVICE)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            zero_shot_probs.append((image_features @ text_features.T).item())
        zero_shot_probs = torch.softmax(torch.tensor(zero_shot_probs), dim=0).numpy()
    
    # Weighted ensemble (70% logistic regression, 30% zero-shot)
    combined_probs = 0.7*probe_pred + 0.3*zero_shot_probs
    return CLASSES[np.argmax(combined_probs)]

## 8. Generate Predictions

Make predictions on the test set and create a submission file.

In [ ]:
# Generate predictions
# Update paths as needed for your environment
test_images = [Path("/kaggle/input/soil-classification/soil_classification-2025/test")/img_id for img_id in test_df.image_id]
print(f"Generating predictions for {len(test_images)} test images...")

# Process images in batches and show progress
predictions = []
for i, img_path in enumerate(test_images):
    if i % 10 == 0:
        print(f"Processing image {i+1}/{len(test_images)}")
    predictions.append(predict_image(img_path))

test_df["soil_type"] = predictions

## 9. Create Submission File

Save the predictions to a CSV file for submission.

In [ ]:
# Save submission file
submission_path = "submission.csv"
test_df.to_csv(submission_path, index=False)
print(f"Submission file saved to {submission_path}")
print(f"Prediction distribution:\n{test_df['soil_type'].value_counts()}")

## 10. Inference Summary

Summarize the inference process and results.

In [ ]:
print("Inference Summary:")
print(f"- Model: {MODEL_NAME} with {PRETRAINED} weights")
print(f"- Test samples: {len(test_df)}")
print(f"- Prediction approach: Hybrid (70% logistic regression, 30% zero-shot CLIP)")
print(f"- Classes: {CLASSES}")
print(f"- Submission file: {submission_path}")